## Scientific Computing
### Assignment 2: NumPy

[Software, Data and
Technology](https://lp.jetbrains.com/software-data-and-technology-constructor-university/) bachelor program at [Constructor
University](https://constructor.university) (Bremen).

In all tasks, the word "array" refers to an object of type `np.ndarray`. Unless otherwise specified, it is forbidden to use loops and list comprehensions!

_Tasks 1, 6, 7 are adapted from HW№2 of [this course](http://wiki.cs.hse.ru/Машинное_обучение_на_матфаке_2018/2019), compiled by Evgeny Kovalev based on materials from the Faculty of Computer Science at the HSE University._

In [4]:
import numpy as np

### Task 1 (1 point)

Write a function `closest_value(array, value)` that calculates the closest and the farthest numbers to the given value in the given array of numbers. For example, if the input is an array `array([0, 1, 2, 3, 4])` and the number `1.33`, the answer will be `(1, 4)`.

**Hint**. You will find the function `np.argmin()` or the method `.argmin()` useful, as well as the function `np.abs()`, which allows you to quickly take the element-wise absolute value.

In [8]:
def closest_value(array, value):
    furth = np.argmax(np.abs(array - value))
    clos = np.argmin(np.abs(array - value))
    return array[clos], array[furth]
    

In [9]:
assert closest_value(np.array([0, 1, 2, 3, 4]), 1.33) == (1, 4)
assert closest_value(np.array([0]), 0) == (0, 0)
assert closest_value(np.array([1, 1.1, 1.11, 1.111, 1.1111]), 100) == (1.1111, 1)
assert closest_value(np.array([-100, 400, 100500]), 0) == (-100, 100500)
print("Task 1 passed")

Task 1 passed


### Task 2 (1 point)
Write a function `wipe_even(arr, target_value, in_place)` that takes as input an array of integers `arr`, and returns an array obtained from `arr` by replacing all even elements with `target_value`. If `target_value` is not specified, it should be considered equal to 0. If the parameter `in_place` is specified and is `True`, then the function should modify the original array, and if it's not specified or set to `False`, it should keep the original array unchanged (but return a modified version according to the condition).

In [10]:
def wipe_even(arr, target_value = 0, in_place = False):
    if in_place == True:
        arr[arr % 2 == 0] = target_value
        return arr.copy()
    return np.where(arr % 2 == 0, target_value, arr)

In [11]:
from timeit import timeit
import numpy as np

def test(inp, outp, target=0, in_place=False):
    inp = np.array(inp)
    inp_backup = np.array(inp)

    q = wipe_even(inp, target, in_place)
    assert isinstance(q, np.ndarray), "Function should return a numpy array (np.array)"
    assert np.array_equal(q, np.array(outp)), "Error for input list "+str(np.array(inp))
    if in_place:
        assert np.array_equal(inp, np.array(outp)), "Function should modify the original list"
    else:
        assert np.array_equal(inp, inp_backup), "Original list should remain unchanged"

test([1, 2, 3, 4, 5], [1, 0, 3, 0, 5], in_place=True)
test([], [], in_place=True)
test([1, 3, 5], [1, 3, 5], in_place=True)
test([5, 4, 3, 2, 0], [5, 0, 3, 0, 0], in_place=True)
test([100, 200, 300, 199, 299, 150], [0, 0, 0,  199, 299, 0], in_place=True)

test([1, 2, 3, 4, 5], [1, 99, 3, 99, 5], target = 99, in_place=True)

N = 100000
benchmark = timeit("[0 if x*2 else x for x in np.array([1]*N)]", 
                   "from __main__ import np, N", number=1)
otherbenchmark = timeit("wipe_even(np.array([1]*N), in_place=True)", 
                        "from __main__ import np, N, wipe_even", number=1)
assert benchmark > otherbenchmark*1.5, "The code works too slowly — are you sure you didn't use loops?"
print("Task 2 passed")

Task 2 passed


### Task 3 (1 point)
Write a function `is_increasing_np(arr)` that checks if the array `arr` is strictly increasing. You are not allowed to use sorting.

**Hint.** You will find function `np.all()` or method `np.ndarray.all()` useful.

In [12]:
def is_increasing_np(arr):
    return np.all(arr[:-1] < arr[1:])

In [13]:
def _is_increasing(arr):
    return is_increasing_np(np.array(arr))

assert _is_increasing([1, 2, 3, 4, 5])
assert _is_increasing([1, 10, 15, 16, 17])
assert _is_increasing([5])
assert not _is_increasing([5, 5])
assert not _is_increasing([5, 6, 7, 7])
assert not _is_increasing([5, 6, 7, 8, 7])
assert not _is_increasing([2, 1, 5, 6])

assert not _is_increasing([1, 1])

assert not _is_increasing([1, 2, 3, 3])
assert _is_increasing(list(range(10000)))
assert not _is_increasing([3, 2, 3])
print("Task 3 passed")

Task 3 passed


### Task 4 (1 point)
Write a function `weighted_sum(weights, grades, normalize)` that returns the weighted sum of grades stored in the `grades` array, according to the weights stored in the `weights` array. For example, for `weights = np.array([0.3, 0.3, 0.4])` and `grades = np.array([7, 9, 8])`, the function should return the number $0.3\times 7+0.3\times 9+0.4\times 8=8.0$.

If the `normalize` parameter is set to `True`, and the sum of all weights is not equal to 1, then all weights should be multiplied by the same number so that their sum equals 1. Otherwise, the weights should be used "as is", even if their sum is not equal to 1. If the function is called without specifying the `normalize` parameter, it should be assumed that `normalize = False`.

In [35]:
def weighted_sum(weights, grades, normalize = False):
    if normalize == True and np.sum(weights) != 1.0:
        sm = np.sum(weights)
        weights = weights * (1.0 / sm)
    return np.sum(grades * weights)

In [36]:
from timeit import timeit
import numpy as np

def test(w, g, out, normalize = False):
    q = weighted_sum(np.array(w), np.array(g), normalize)
    assert np.isclose(q, out)

test([0.3, 0.3, 0.4], [7, 9, 8], 8)
test([0.1, 0.2, 0.3, 0.4], [1, 5, 3, 2], 2.8)
test([1, 2, 3, 4], [1, 5, 3, 2], 28)
test([1, 2, 3, 4], [1, 5, 3, 2], 2.8, normalize=True)

In [37]:
N = 1000000

test([1, 2, 3, 4], [1, 5, 3, 2], 28)

benchmark = timeit("sum([x/x for x in np.array([1]*N)])", "from __main__ import N, np", number=1)
otherbenchmark = timeit("weighted_sum(np.array([1.1]*N), np.array([1]*N), True)", 
                        "from __main__ import N, weighted_sum, np", number=1)
assert benchmark > otherbenchmark*1.4, "The code works too slowly — are you sure you used numpy methods?"

print("Task 4 passed")

Task 4 passed


### Task 5 (1 point)
Write a function `find_max(matrix)` that takes a matrix `matrix` as input and returns an array whose elements are the maximum values of the columns of `matrix`. (Hint: various NumPy functions take `axis` argument.)

In [68]:
def find_max(matrix):
    return matrix[np.argmax(matrix, axis = 0), np.arange(matrix.shape[1])]

In [69]:
assert (find_max(np.array([[9, 9, 4], [8, 8, 1], [5, 3, 6], [3, 3, 3], [2, 1, 9]])) == np.array([9, 9, 9])).all()
assert (find_max(np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])) == np.array([6, 7, 8])).all()
assert (find_max(np.array([[0, 1, 2], [3, 0, 5], [6, 7, 1]])) == np.array([6, 7, 5])).all()
assert (find_max(np.array([[-10, 1, 2, 5], [3, 30, 5, 17], [-100, -100, -100, 100], [1, 1, 2, -80]])) == 
        np.array([3, 30, 5, 100])).all()
assert (find_max(np.array([[1]])) == np.array([1])).all()

print("Task 5 passed")

Task 5 passed


### Task 6 (1 point)

Write a function `diag_prod(matrix)` that calculates the product of all non-zero diagonal elements on the diagonal of a given square matrix. For example, if the input matrix is
$$
\begin{pmatrix}
0 & 1 & 2\\
3 & 4 & 5\\
6 & 7 & 8\\
\end{pmatrix},
$$
then the answer will be 32. It is guaranteed that there will be at least one non-zero element on the diagonal of the matrix.

**Hint.** Functions that may be useful in solving: `np.diag()`, `np.prod()`

In [74]:
def diag_prod(matrix):
    return np.prod(np.diag(np.where(matrix == 0, 1, matrix)))

In [76]:
assert diag_prod(np.array([[0, 1, 2], [3, 4, 5], [6, 7, 8]])) == 32
assert diag_prod(np.array([[0, 1, 2], [3, 0, 5], [6, 7, 1]])) == 1
assert diag_prod(np.array([[-10, 1, 2, 5], [3, 30, 5, 17], [-100, -100, -100, 100], [1, 1, 2, -80]])) == -2400000
assert diag_prod(np.array([[1]])) == 1

print("Task 6 passed")

Task 6 passed


### Task 7 (1 point)

Write a function `make_symmetric(matrix)` that makes the given [triangular matrix](https://en.wikipedia.org/wiki/Triangular_matrix) symmetric. For example, if the input matrix is
$$
\begin{pmatrix}
1 & 2 & 3 & 4\\
0 & 5 & 6 & 7\\
0 & 0 & 8 & 9\\
0 & 0 & 0 & 10\\
\end{pmatrix},
$$
then the output should be the matrix
$$
\begin{pmatrix}
1 & 2 & 3 & 4\\
2 & 5 & 6 & 7\\
3 & 6 & 8 & 9\\
4 & 7 & 9 & 10\\
\end{pmatrix}.
$$
**Hint.** `np.diag` can not only get the diagonal from an existing matrix but also create a matrix with a given diagonal.

In [77]:
def make_symmetric(matrix):
    return matrix + matrix.T - np.diag(np.diag(matrix))

In [78]:
assert (make_symmetric(np.array([[1, 2, 3, 4], [0, 5, 6, 7], [0, 0, 8, 9], [0, 0, 0, 10]])) ==
        np.array([[1, 2, 3, 4], [2, 5, 6, 7], [3, 6, 8, 9], [4, 7, 9, 10]])).all()
assert (make_symmetric(np.array([[0, 0, 0, 0], [0, 9, 8, 0], [0, 0, -100, -1000], [0, 0, 0, 999]])) ==
        np.array([[0, 0, 0, 0], [0, 9, 8, 0], [0, 8, -100, -1000], [0, 0, -1000, 999]])).all()
assert (make_symmetric(np.array([[1]])) == np.array([[1]])).all()
print("Task 7 passed")

Task 7 passed


    ### Task 8 (1 point)
Write a function `mean_by_group(grades, groups)` that takes as input two arrays of the same length: the `grades` array contains the grades of some students, and the `groups` array contains their group as strings `A` or `B`. The function should return a dictionary where the keys are the strings `A` and `B`, and the values are the arithmetic mean of the grades for students of the corresponding group.

For example, if `grades = np.array([5, 4, 3, 5, 2])` and `groups = np.array(["B", "A", "A", "B", "A"])`, the function should return the dictionary `{'A': 3.0, 'B': 5.0}`.

**Hint.** For quick calculation of the mean, there is the `np.mean()` function or the corresponding method for objects of type `numpy.ndarray`.

In [87]:
def mean_by_group(grades, groups):
    return{'A':np.mean(grades[groups == 'A']), 'B':np.mean(grades[groups == 'B'])}

In [88]:
from timeit import timeit
import numpy as np

def test(grades, groups, outp):
    ret = mean_by_group(np.array(grades), np.array(groups))
    assert np.isclose(ret['A'], outp['A'])
    assert np.isclose(ret['B'], outp['B'])

test([5, 4, 3, 5, 2], ["B", "A", "A", "B", "A"], {'A': 3.0, 'B': 5.0})
test([1, 0]*10, ['B', 'A']*10, {'B': 1, 'A': 0})
test(range(100), ['B', 'A']* 50, {'B': 49.0, 'A': 50.0})
test(list(range(100))+[100], ['A']*100+['B'], {'A':49.5, 'B': 100.0})

def benchmark_test(a, b):
    xx = 0
    yy = 0
    im = 0
    fi = 0
    for x, y in zip(a, b):
        if x != y:
            xx += x
            yy += x
            im += 1
            fi += 1

    return xx+yy

N = int(1E5)
grades = np.array([1.1]*N + [2.2]*N)
groups = np.array(['A']*N + ['B']*N)

benchmark = timeit("assert np.isclose(mean_by_group(grades, groups)['A'], 1.1)",
                   "from __main__ import np, mean_by_group, grades, groups",
                   number=1)
reference_benchmark = timeit("benchmark_test(grades, groups)",
                             "from __main__ import benchmark_test, grades, groups",
                             number=1)

assert reference_benchmark > benchmark * 10, "The code works too slowly - are you sure you used numpy methods?"

print("Task 8 passed")

Task 8 passed


### Task 9 (2 points)

In a certain kingdom, in a certain state, the personal income tax is calculated as follows. The basic tax rate is 13%. (It's a fairy tail, so it's unrealistically low.) If in any month your earnings for the year exceed one thousand tugrics, then for the rest of the year (not including that month) a rate of 20% is established. For example, if you earn 150 tugrics every month, then by July you will have earned $150\times 7 = 1050$ tugrics and starting from August, income tax will be charged at a rate of 20%. Write a function `calculate_tax(income)` that takes as input an array containing income for each month of the year, starting from the first, and returns the total amount of tax to be paid for the year. The year in a certain kingdom can last more than 12 months if a corresponding highest decree is adopted on this matter.

**Hint.** The functions `np.cumsum()` and `np.where()` will help you.

In this task, you can use `if` exactly once.

In [104]:
def calculate_tax(income):
    sm = np.cumsum(income)
    if (sm[sm > 1000].size == 0):
        return sm[-1] * 0.13
    return sm[-1] * 0.13 + (sm[sm>1000][-1] - sm[sm > 1000][0]) * 0.07

In [105]:
from timeit import timeit
import numpy as np

assert np.isclose(calculate_tax(np.array([150]*12)), 286.5)
assert np.isclose(calculate_tax(np.array([100]*12)), 163)
assert np.isclose(calculate_tax(np.array([50]*12)), 78)
assert np.isclose(calculate_tax(np.array([1000]*12)), 2260)

assert np.isclose(calculate_tax(np.array(range(12))*100), 1215)
assert np.isclose(calculate_tax(np.array(range(11,-1,-1))*100), 1243)

In [106]:
def dummy(x):
    z = 0
    for y in x:
        z += y
        z += y*0.12
        if z:
            z += y
    return z

assert np.isclose(calculate_tax(np.array(range(12))*100), 1215)

N = int(1E6)
arr = np.array([1]*N)
benchmark = timeit("calculate_tax(arr)", "from __main__ import calculate_tax, arr", number=1)
reference_benchmark = timeit("dummy(arr)", "from __main__ import dummy, arr", number=1)

assert reference_benchmark > benchmark*5, "The code is running too slowly - are you sure you used numpy methods?"

print("Task 9 passed")

Task 9 passed



### Task 10 (1 point)

In machine learning tasks, it is often necessary to normalize data before using it. Let's say the variable `X` contains a two-dimensional `np.array`, where different objects are recorded in rows and features in columns. You need to write a function `normalize(X)` that takes the array `X` as input and normalizes all variables so that their mean is 0 and standard deviation is 1. In other words, for each column, you need to subtract the mean of that column from all elements and divide the result by the standard deviation of that column. More formally: if $X=(x_{ij})$ is our matrix, $x_{ij}$ is the element in its $i$-th row and $j$-th column, and $x_{\cdot j}$ is the $j$-th column, then in the new matrix, the element in the $i$-th row and $j$-th column will be:

$$\widehat{x}_{ij}=\frac{x_{ij}-\overline{x_{\cdot j}}}{\sigma_{x_{\cdot j}}},$$
where $\overline{x_{\cdot j}}$ is the sample mean (arithmetic mean) of all elements in the $j$-th column, $\sigma_{x_{\cdot j}}$ is the standard deviation of all elements in the $j$-th column.

**Hint:** You can calculate the mean using the `.mean()` method and the standard deviation using `.std()`.  The task can be solved in one line.

In [107]:
def normalize(X):
    return (X - np.mean(X, axis=0)) / np.std(X, axis=0)

In [108]:
assert np.isclose(normalize(np.array([[ 1.00766597, -1.1201796 ,  2.47274732, -0.33619288,  1.50555214],
       [ 1.48986823,  0.80894409,  0.55980545,  0.67813423, -0.3187493 ]])), np.array([[-1., -1.,  1., -1.,  1.],
       [ 1.,  1., -1.,  1., -1.]])).all()
assert np.isclose(normalize(np.array([[-0.98607026],
       [ 1.93312384],
       [-0.99905497],
       [-0.95934573],
       [ 0.05295053]])), np.array([[-0.69959273],
       [ 1.87124093],
       [-0.71102792],
       [-0.67605736],
       [ 0.21543708]])).all()
assert np.isclose(normalize(np.array([[-1.63419424],
       [ 0.39451389],
       [-0.11346483],
       [ 0.56117231],
       [ 0.35460207],
       [ 1.50836012],
       [ 0.5176692 ],
       [-1.20605276],
       [ 0.7904588 ],
       [ 1.28349441]])), np.array([[-1.9874883 ],
       [ 0.15738144],
       [-0.37968359],
       [ 0.33358254],
       [ 0.11518431],
       [ 1.33500529],
       [ 0.28758849],
       [-1.53483191],
       [ 0.57599773],
       [ 1.09726401]])).all()
assert np.isclose(normalize(np.array([[-1.31158329,  2.5954087 , -1.01662736, -0.27565263,  0.52639556,
         0.58218805, -0.35961103,  0.31096071,  0.52193677, -0.41754881],
       [-0.19218836, -0.03416295,  0.80408723, -1.18733572,  0.14422448,
         0.6091103 ,  0.67617586,  0.17732224,  0.99660189, -0.07798097]])), np.array([[-1.,  1., -1.,  1.,  1., -1., -1.,  1., -1., -1.],
       [ 1., -1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1.]])).all()
assert np.isclose(normalize(np.array([[-0.28368534, -0.90928588, -1.35180963],
       [ 1.30199557,  1.32081835,  1.11951334]])), np.array([[-1., -1., -1.],
       [ 1.,  1.,  1.]])).all()
assert np.isclose(normalize(np.array([[-0.34089722,  0.93727935],
       [ 0.14410815, -0.96321317],
       [-1.98355493, -0.0310602 ]])), np.array([[ 0.42383229,  1.23244371],
       [ 0.95653353, -1.21689804],
       [-1.38036582, -0.01554567]])).all()
assert np.isclose(normalize(np.array([[ 1.53033913,  0.05456373,  0.22504087, -1.16687133, -0.23619502],
       [-0.81477156,  1.96405223, -1.5506048 , -2.08082958, -0.23459537],
       [-0.80961303, -0.55950949, -1.07953561,  0.571387  , -1.03341414],
       [ 0.10526012, -2.06172783, -1.1661957 , -1.00297227, -1.02432731],
       [ 0.04661   , -0.21104596, -0.84339233,  0.22806353, -0.34655384]])), np.array([[ 1.77181211,  0.16828692,  1.84979571, -0.49193269,  0.90886342],
       [-0.96400966,  1.64710001, -1.11468608, -1.43524085,  0.91315434],
       [-0.95799169, -0.30728521, -0.32822509,  1.30214625, -1.22961355],
       [ 0.10930543, -1.47068586, -0.47290614, -0.32277035, -1.20523884],
       [ 0.04088381, -0.03741586,  0.0660216 ,  0.94779764,  0.61283463]])).all()

print("Task 10 passed")

Task 10 passed


### Task 11 (2 points)
In a two-dimensional array `scores`, the scores of several students are recorded, with each row representing a student and each column representing a homework assignment. There is also an array `max_scores`, which contains as many elements as there are columns in `scores`: it lists the maximum number of points that could be earned for the corresponding homework assignment. Theoretically, a student could solve problems for more points, but points earned above the maximum are not counted. The grade for a homework assignment is a floating-point number from 0 to 10 and is determined as *points earned / maximum number of points × 10*. For example, if the maximum number of points for a homework assignment is 8, and a student earned 4 points for it, then the grade is *4/8×10=5*. And if they had earned 12 points, only 8 points would be counted, and the grade would be 10. The course grade is calculated as the arithmetic mean of all homework grades, rounded to the nearest integer using the `np.round` function. Write a function `get_grades(scores, max_scores)` that returns an array of final grades.

In [111]:
def get_grades(scores, max_scores):
    
    return np.round(np.mean(np.minimum(scores, max_scores) / max_scores * 10, axis=1))

In [112]:
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([1, 1])), np.array([ 10.,  10.,  10.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([1, 2])), np.array([ 10.,  10.,  10.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([1, 3])), np.array([  8.,  10.,  10.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([1, 6])), np.array([  7.,   8.,  10.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([ 1, 10])), np.array([ 6.,  7.,  8.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([2, 1])), np.array([  8.,  10.,  10.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([2, 2])), np.array([  8.,  10.,  10.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([2, 3])), np.array([  6.,  10.,  10.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([2, 6])), np.array([  4.,   8.,  10.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([ 2, 10])), np.array([ 4.,  7.,  8.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([3, 1])), np.array([  7.,  10.,  10.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([3, 2])), np.array([  7.,  10.,  10.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([3, 3])), np.array([  5.,  10.,  10.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([3, 6])), np.array([  3.,   8.,  10.])).all()
assert np.isclose(get_grades(np.array([[1, 2],
       [3, 4],
       [5, 6]]), np.array([ 3, 10])), np.array([ 3.,  7.,  8.])).all()
assert np.isclose(get_grades(np.array([[  9.,   9.,  10.],
       [  1.,   9.,   0.],
       [  1.,   3.,  10.],
       [  5.,   5.,   2.],
       [  3.,   9.,   3.]]), np.array([ 9.,  9.,  2.])), np.array([ 10.,   4.,   5.,   7.,   8.])).all()
assert np.isclose(get_grades(np.array([[  8.,   3.,   5.,  10.,   4.],
       [  9.,   0.,   5.,  10.,   6.],
       [  0.,   1.,   7.,   2.,   9.]]), np.array([ 9.,  3.,  3.,  2.,  6.])), np.array([ 9.,  8.,  7.])).all()
assert np.isclose(get_grades(np.array([[ 6.,  4.,  2.,  7.,  0.],
       [ 8.,  1.,  4.,  4.,  8.],
       [ 1.,  3.,  5.,  5.,  3.],
       [ 2.,  5.,  3.,  4.,  8.],
       [ 7.,  0.,  7.,  1.,  8.]]), np.array([ 5.,  5.,  8.,  3.,  8.])), np.array([ 6.,  7.,  6.,  8.,  6.])).all()

print("Task 11 passed")

Task 11 passed
